In [ ]:
# --- 必須ライブラリのインストール ---
!pip install -U transformers accelerate peft bitsandbytes scipy numpy pandas

# --- 2. QLoRA環境のセットアップ ---
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model

# 1. デバイスの確認 (GPU割り当て)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\n--- デバイス情報 ---")
print(f"使用デバイス: {device}")

# 2. QLoRA (4ビット量子化) の設定
# GPUメモリを節約するための設定
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# 3. LLMとTokenizerのロード
# 🚨 認証不要な代替モデルに変更 🚨
model_id = "mistralai/Mistral-7B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id)

# 量子化設定を適用してモデルをロード
print(f"\n--- モデルロード中 ({model_id}) ---")
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

# 4. LoRA設定オブジェクトの定義 (PEFT設定)
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"], # MistralのAttention層に適したターゲット
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

# 5. PEFTモデルの準備
model = get_peft_model(model, lora_config)

# 最終確認: 学習可能なパラメータ数が劇的に少ないことを確認
print("\n--- LoRA適用後のパラメータ確認 ---")
model.print_trainable_parameters()


--- デバイス情報 ---
使用デバイス: cuda


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]


--- LoRA適用後のパラメータ確認 ---
trainable params: 13,631,488 || all params: 7,255,363,584 || trainable%: 0.1879
